# 국가별 알콜 섭취량 데이터 분석
- 데이터 읽기
- 데이터 정보 확인
- 컬럼명 한글로 변경
    - 원본 데이터 컬럼 정보
    - country : 국가명
    - beer_servings : 맥주 소비량
    - spirit_servings : 증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)
    - wine_servings : 와인 소비량
    - total_litres_of_pure_alcohol : 순수 알코올
    - continent : 대륙
- 결측치 확인
    - `대륙`에 대한 정보가 부족한 것을 확인할 수 있다.
    - drinks.csv 파일을 열어 원인을 확인한다.
    - 대륙의 종류
        - 아시아, 유럽, 아프리카, 북아메리카, 남아메리카, 오세아니아
        - AS, EU, AF, NA, SA, OC
- 결측치 해결
    - 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv 함수의 na_filter=False 파라메터 : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 
- 통계값 구하기
    - 전세계 맥주, 증류주, 와인, 알코올의 소비에 대한 '평균', '최대값', '최소값' 구하기
- 주류 소비량 비교
    - 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬
    - 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색
    - 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색
    - 맥주를 가장 많이 소비하는 국가 확인
    - 증류주를 가장 많이 소비하는 국가 확인
    - 평균 맥주 소비량이 가장 많은 대륙 확인
- 데이터 프레임 조작
    - 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가
    - 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
        - 알코올비율 = '알코올' / '주류소비량'
        - 알코올비율'이 NaN인 것에 대해 0으로 채우기
    - 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회
    - DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장
    - 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기
        <pre>
               맥주	    증류주	     와인	      알코올	  주류소비량	알코올비율
     국가						
South Korea	140.000000	16.000000	9.000000	9.800000	165.000000	0.059394
World Wide	106.160622	80.994819	49.450777	4.717098	236.606218	0.026115
</pre>
    - pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구한다.

## 데이터 읽기
csv 파일 가져오기 
- csv 파일은 컬럼 구분이 콤마(,) 로 이루어진 파일
- df = pd.read_csv(파일이름) : csv 형식의 파일을 읽어 DataFrame 객체로 가져옴
- DataFrame.to_csv(파일이름, index=True) : DataFrame을 csv 파일로 저장, index는 제외하고 저장하기 위해서는 index=False로 지정

In [1]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('./drinks.csv')
print(df.shape)
print(df.head())

(193, 6)
       country  beer_servings  spirit_servings  wine_servings  \
0  Afghanistan              0                0              0   
1      Albania             89              132             54   
2      Algeria             25                0             14   
3      Andorra            245              138            312   
4       Angola            217               57             45   

   total_litres_of_pure_alcohol continent  
0                           0.0        AS  
1                           4.9        EU  
2                           0.7        AF  
3                          12.4        EU  
4                           5.9        AF  


## 컬럼명 한글로 변경

In [11]:
### 결측치 확인
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   국가      193 non-null    object 
 1   맥주      193 non-null    int64  
 2   증류주     193 non-null    int64  
 3   와인      193 non-null    int64  
 4   알코올     193 non-null    float64
 5   대륙      170 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


In [8]:
### 일부 컬럼명 변경 
df.rename({'country':'국가'}, axis = 1, inplace = True)

In [10]:
### 전체 컬럼 다 바꾸기 -> df.columns
df.columns = ['국가','맥주','증류주','와인','알코올','대륙']

## 결측치 확인

In [41]:
print(df.isna().sum())

국가명                                            0
와인 소비량                                         0
증류주 소비량(독주, 알코올 함량 20% 이상의 설탕을 첨가하지 않은 증류주)    0
와인 소비량                                         0
순수 알코올                                         0
대륙                                             0
dtype: int64


## 결측치 해결

- 'NA'를 NaN(결측치)로 읽는 것을 방지
    - pd.read_csv(파일이름, na_filter=False) : 결측치를 읽어올지 여부 지정
    - 옵션 확인 : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 


- 결측치를 다른 값으로 채우기 함
    - DataFrame.fillna(값) 또는 Series.fillna(값) : 결측값을 특정 값으로 채우기 
    - 연속형 데이터 : 0으로 채우기, 평균값으로 채우기, 범주별 평균값 채우기 등
    - 범주형 데이터 : 다른 범주로 만들어 채우기 

In [12]:
# 해당 결측치를 가지고 있는 unique
print(df['대륙'].unique())

['AS' 'EU' 'AF' nan 'SA' 'OC']


In [14]:
# 결측치를 가지고 있는 국가명
print(df[df['대륙'].isna()]['국가'])

5                 Antigua & Barbuda
11                          Bahamas
14                         Barbados
17                           Belize
32                           Canada
41                       Costa Rica
43                             Cuba
50                         Dominica
51               Dominican Republic
54                      El Salvador
68                          Grenada
69                        Guatemala
73                            Haiti
74                         Honduras
84                          Jamaica
109                          Mexico
122                       Nicaragua
130                          Panama
143               St. Kitts & Nevis
144                       St. Lucia
145    St. Vincent & the Grenadines
174               Trinidad & Tobago
184                             USA
Name: 국가, dtype: object


In [17]:
# 첫번째 방법
temp = pd.read_csv('./drinks.csv', na_filter = False)
print(temp.isna().sum())

country                         0
beer_servings                   0
spirit_servings                 0
wine_servings                   0
total_litres_of_pure_alcohol    0
continent                       0
dtype: int64


In [18]:
# 두번째 방법 (fillna : 모든 결측치 값을 괄호 안 데이터로 수정)
df = df.fillna('NA')
print(df)
print(df.isna().sum())

              국가   맥주  증류주   와인   알코올  대륙
0    Afghanistan    0    0    0   0.0  AS
1        Albania   89  132   54   4.9  EU
2        Algeria   25    0   14   0.7  AF
3        Andorra  245  138  312  12.4  EU
4         Angola  217   57   45   5.9  AF
..           ...  ...  ...  ...   ...  ..
188    Venezuela  333  100    3   7.7  SA
189      Vietnam  111    2    1   2.0  AS
190        Yemen    6    0    0   0.1  AS
191       Zambia   32   19    4   2.5  AF
192     Zimbabwe   64   18    4   4.7  AF

[193 rows x 6 columns]
국가     0
맥주     0
증류주    0
와인     0
알코올    0
대륙     0
dtype: int64


In [19]:
# 대륙별 레코드 개수
print(df['대륙'].value_counts())

대륙
AF    53
EU    45
AS    44
NA    23
OC    16
SA    12
Name: count, dtype: int64


## 통계값 구하기
- DataFrame.describe() : 컬럼별 count, mean, std, min, max, Q1, Q2, Q3 등의 통계값 구하기

In [20]:
# describe() 함수는 기본적으로 연속형(수치형) 데이터에 대한 통계를 보여준다.
df.describe()

,맥주,증류주,와인,알코올
count,193.000000,193.000000,193.000000,193.000000
mean,106.160622,80.994819,49.450777,4.717098
std,101.143103,88.284312,79.697598,3.773298
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,1.000000,1.300000
50%,76.000000,56.000000,8.000000,4.200000
75%,188.000000,128.000000,59.000000,7.200000
max,376.000000,438.000000,370.000000,14.400000


In [21]:
# 데이터 타입에 상관 없이 모든 컬럼에 대한 통계값 확인
# count : 결측치를 포함하지 않는 레코드 개수
# unique : 종류별 하나
# top : 가장 많은 빈도수를 갖는 데이터
print(df.describe(include='all'))

                 국가          맥주         증류주          와인         알코올   대륙
count           193  193.000000  193.000000  193.000000  193.000000  193
unique          193         NaN         NaN         NaN         NaN    6
top     Afghanistan         NaN         NaN         NaN         NaN   AF
freq              1         NaN         NaN         NaN         NaN   53
mean            NaN  106.160622   80.994819   49.450777    4.717098  NaN
std             NaN  101.143103   88.284312   79.697598    3.773298  NaN
min             NaN    0.000000    0.000000    0.000000    0.000000  NaN
25%             NaN   20.000000    4.000000    1.000000    1.300000  NaN
50%             NaN   76.000000   56.000000    8.000000    4.200000  NaN
75%             NaN  188.000000  128.000000   59.000000    7.200000  NaN
max             NaN  376.000000  438.000000  370.000000   14.400000  NaN


In [23]:
# 보고 싶은 컬럼만 추출하는 방법도 있음
df2 = df[['맥주','증류주','와인','알코올']]
print(df2.agg(['min','max','mean'])) # 일부 컬럼의 일부 집계량만 추출 : agg

              맥주         증류주          와인        알코올
min     0.000000    0.000000    0.000000   0.000000
max   376.000000  438.000000  370.000000  14.400000
mean  106.160622   80.994819   49.450777   4.717098


## 주류 소비량 비교



### 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬 (Boolean indexing)

In [24]:
result = df[df['와인']>(df['맥주']+df['증류주'])].sort_values('대륙')
print(result)

                      국가   맥주  증류주   와인   알코올  대륙
55     Equatorial Guinea   92    0  233   5.8  AF
148  Sao Tome & Principe   56   38  140   4.2  AF
92                  Laos   62    0  123   6.2  AS
171          Timor-Leste    1    1    4   0.1  AS
61                France  127  151  370  11.8  EU
83                 Italy   85   42  237   6.5  EU
136             Portugal  194   67  339  11.0  EU
6              Argentina  193   25  221   8.3  SA
185              Uruguay  115   35  220   6.6  SA


### 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색 (np.logical_and)

In [26]:
result = df[np.logical_and(df['맥주']>230, df['와인']>230)]
print(result)

             국가   맥주  증류주   와인   알코올  대륙
3       Andorra  245  138  312  12.4  EU
99   Luxembourg  236  133  271  11.4  EU
156    Slovenia  270   51  276  10.6  EU


### 전체 평균보다 많은 알코올을 섭취하는 대륙과 알코올 평균 소비량 검색 (중요)

In [32]:
# 전체 알코올 평균 섭취량
total_mean = df['알코올'].mean()
print('전체 알코올 평균 섭취량',total_mean)
print()
# 대륙별 알코올 평균 섭취량
conti_mean = df.groupby('대륙')['알코올'].mean()
print('대륙별 알코올 평균 섭취량', conti_mean)
print()
# 전체 평균보다 많은 알코올을 섭취하는 대륙
print('전체 평균보다 많은 알코올을 섭취하는 대륙')
conti_mean_over = conti_mean[conti_mean > total_mean]
print(conti_mean_over)

전체 알코올 평균 섭취량 4.717098445595855

대륙별 알코올 평균 섭취량 대륙
AF    3.007547
AS    2.170455
EU    8.617778
NA    5.995652
OC    3.381250
SA    6.308333
Name: 알코올, dtype: float64

전체 평균보다 많은 알코올을 섭취하는 대륙
대륙
EU    8.617778
NA    5.995652
SA    6.308333
Name: 알코올, dtype: float64


### 맥주를 가장 많이 소비하는 국가 확인 (Boolean indexing)

In [33]:
result = df[df['맥주'].max() == df['맥주']][['국가','맥주']]
print(result)

          국가   맥주
117  Namibia  376


### 증류주를 가장 많이 소비하는 국가 확인

In [34]:
result = df[df['증류주'].max() == df['증류주']][['국가','증류주']]
print(result)

         국가  증류주
68  Grenada  438


In [35]:
# 맥주 소비량이 많은 상위 5개 국가
result = df.sort_values('맥주', ascending = False).head(5)[['국가','맥주']]
print(result)

                 국가   맥주
117         Namibia  376
45   Czech Republic  361
62            Gabon  347
65          Germany  346
98        Lithuania  343


### 평균 맥주 소비량이 가장 많은 대륙 확인

In [38]:
beer_avg_conti = df.groupby('대륙')['맥주'].mean().sort_values(ascending = False).head(1)
print(beer_avg_conti)

대륙
EU    193.777778
Name: 맥주, dtype: float64


## 데이터 프레임 조작

### 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량') 추가

In [41]:
# 첫번째 방법
# df['주류소비량'] = df['맥주'] + df['증류주'] + df['와인']
# 두번째 방법
# df['주류소비량'] = df[['맥주','증류주','와인']].sum(axis =1)
# 세번째 방법
df['주류소비량'] = df[df.columns[1:4]].sum(axis=1)
df.head()

,국가,맥주,증류주,와인,알코올,대륙,주류소비량
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319


### 주류 소비량 대비 알코올 소비량 비율을 구해 새로운 컬럼 ('알코올비율') 추가
- 알코올비율 = '알코올' / '주류소비량'
- 알코올비율'이 NaN인 것에 대해 0으로 채우기

In [43]:
df['알코올비율'] = df['알코올'] / df['주류소비량']
print(df.head(3))

            국가  맥주  증류주  와인  알코올  대륙  주류소비량     알코올비율
0  Afghanistan   0    0   0  0.0  AS      0       NaN
1      Albania  89  132  54  4.9  EU    275  0.017818
2      Algeria  25    0  14  0.7  AF     39  0.017949


In [44]:
# 주류소비량 결측치 채우기
df['알코올비율'] = df['알코올비율'].fillna(0)
print(df.head(3))

            국가  맥주  증류주  와인  알코올  대륙  주류소비량     알코올비율
0  Afghanistan   0    0   0  0.0  AS      0  0.000000
1      Albania  89  132  54  4.9  EU    275  0.017818
2      Algeria  25    0  14  0.7  AF     39  0.017949


## 맥주 소비량

### 알코올비율이 높은 국가(술을 독하게 마시는 국가) TOP5의 ['국가', '주류소비량', '알코올비율'] 정보 조회

In [46]:
result = df[['국가','주류소비량','알코올비율']].sort_values('알코올비율', ascending = False).head(5)
print(result)
# 인덱스 값을 순위 형태로 바꿈
result.index = np.arange(1,6)
print(result)

               국가  주류소비량     알코올비율
63         Gambia      9  0.266667
153  Sierra Leone     30  0.223333
124       Nigeria     49  0.185714
179        Uganda     54  0.153704
142        Rwanda     45  0.151111
             국가  주류소비량     알코올비율
1        Gambia      9  0.266667
2  Sierra Leone     30  0.223333
3       Nigeria     49  0.185714
4        Uganda     54  0.153704
5        Rwanda     45  0.151111


### DataFrame의 index를 ['대륙', '국가'] 로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장

In [48]:
# sort_index : 인덱스 정렬
df2 = df.set_index(['대륙','국가']).sort_index()
print(df2.head())

                  맥주  증류주  와인  알코올  주류소비량     알코올비율
대륙 국가                                              
AF Algeria        25    0  14  0.7     39  0.017949
   Angola        217   57  45  5.9    319  0.018495
   Benin          34    4  13  1.1     51  0.021569
   Botswana      173   35  35  5.4    243  0.022222
   Burkina Faso   25    7   7  4.3     39  0.110256


### 전세계 평균 소비량과 한국의 소비량을 비교하는 DataFrame 만들기

In [49]:
# case = False : 대문자 소문자 상관 x 
result = df[df['국가'].str.contains('korea', case = False)]
print(result)

              국가   맥주  증류주  와인  알코올  대륙  주류소비량     알코올비율
46   North Korea    0    0   0  0.0  AS      0  0.000000
138  South Korea  140   16   9  9.8  AS    165  0.059394


In [50]:
korea = df[df['국가']=='South Korea']
print(korea)

              국가   맥주  증류주  와인  알코올  대륙  주류소비량     알코올비율
138  South Korea  140   16   9  9.8  AS    165  0.059394


In [ ]:
# 대륙 컬럼 삭제
korea.drop('대륙', axis = 1, inplace = True)

In [54]:
print(korea)

              국가   맥주  증류주  와인  알코올  주류소비량     알코올비율
138  South Korea  140   16   9  9.8    165  0.059394


In [59]:
df.columns[1:7]

Index(['맥주', '증류주', '와인', '알코올', '대륙', '주류소비량'], dtype='object')

In [71]:
# korea 데이터프레임을 만들었기 때문에 world_wide 데이터프레임도 만들어서 합치기
world_wide = pd.DataFrame(df[df.columns[[1,2,3,4,6,7]]].mean())
world_wide = world_wide.T
print(world_wide)

           맥주        증류주         와인       알코올       주류소비량     알코올비율
0  106.160622  80.994819  49.450777  4.717098  236.606218  0.026115


In [72]:
# world_wide 데이터프레임에 '국가' 컬럼이 없으므로 insert (특정 위치에 컬럼 추가)로 컬럼 추가
world_wide.insert(0, '국가', 'World Wide')
print(world_wide)

           국가          맥주        증류주         와인       알코올       주류소비량  \
0  World Wide  106.160622  80.994819  49.450777  4.717098  236.606218   

      알코올비율  
0  0.026115  


In [73]:
# concat으로 데이터프레임 두개 연결
compare = pd.concat([korea, world_wide])
print(compare)

              국가          맥주        증류주         와인       알코올       주류소비량  \
138  South Korea  140.000000  16.000000   9.000000  9.800000  165.000000   
0     World Wide  106.160622  80.994819  49.450777  4.717098  236.606218   

        알코올비율  
138  0.059394  
0    0.026115  


In [74]:
# '국가' 컬럼을 index로 만들기
compare = compare.set_index('국가') 
print(compare)

                     맥주        증류주         와인       알코올       주류소비량     알코올비율
국가                                                                           
South Korea  140.000000  16.000000   9.000000  9.800000  165.000000  0.059394
World Wide   106.160622  80.994819  49.450777  4.717098  236.606218  0.026115


### pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구한다.

In [75]:
result = pd.pivot_table(data= df, index='대륙', values = ['맥주','와인'], aggfunc = ['mean','median','max'])
print(result)

          mean             median         max     
            맥주          와인     맥주     와인   맥주   와인
대륙                                                
AF   61.471698   16.264151   32.0    2.0  376  233
AS   37.045455    9.068182   17.5    1.0  247  123
EU  193.777778  142.222222  219.0  128.0  361  370
NA  145.434783   24.521739  143.0   11.0  285  100
OC   89.687500   35.625000   52.5    8.5  306  212
SA  175.083333   62.416667  162.5   12.0  333  221
